# Исследование надёжности заёмщиков – анализ банковских данных

# Цель:

Исследовать влияние семейного положения, количество детей клиента на факт погашения кредита в срок. Результаты работы будут использованы при построении модели кредитного скоринга – системы для оценки платежеспособности потенциального заемщика.

# Описание данных:


•	`children` — количество детей в семье

•	`days_employed` — общий трудовой стаж в днях

•	`dob_years` — возраст клиента в годах

•	`education` — уровень образования клиента

•	`education_id` — идентификатор уровня образования

•	`family_status` — семейное положение

•	`family_status_id` — идентификатор семейного положения

•	`gender` — пол клиента

•	`income_type` — тип занятости

•	`debt` — имел ли задолженность по возврату кредитов

•	`total_income` — ежемесячный доход

•	`purpose` — цель получения кредита


# Оглавление:

1. [Предобработка  данных:](#part_0)

    - [Обработка пропущенные значения](#part_1)

    - [Обработка дубликаты](#part_2)

    - [Обработка артефактов в данных](#part_3)

    - [Лемматизация](#part_4)

    - [Категоризация данных](#part_5)

2. [Исследовательский анализ данных:](#part_6)

    - [Зависимость между наличием детей и возвратом кредита в срок](#part_7)

    - [Зависимость между семейным положением и возвратом кредита в срок](#part_8)

    - [Зависимость между уровнем дохода и возвратом кредита в срок](#part_9)

    - [Влияние целей кредита на его возврат в срок](#part_10)

3. [Вывод и рекомендации для построения модели кредитного скоринга](#part_11)


## Предобработка данных

### Обзор данных:

In [1]:
# Получим доступ к библиотеке Pandas, используем имя переменной pd

import pandas as pd

In [2]:
# Чтобы скрыть предупреждия от Jupiter Notebook импортируем библиотеку warnings


import warnings

warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/datasets/data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [5]:
df.isna().sum() # сумма пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# Доля пропусков

(df['days_employed'].isna().sum() + df['total_income'].isna().sum()) / len(df)

0.20199767711962835

In [7]:
df.duplicated().sum() # сумма дубликатов

54

In [8]:
# Доля дубликатов

(df.duplicated().sum()) / len(df) 

0.0025087108013937282

In [9]:
#  Посмотрим уникальные значения для типа занятости 

df['income_type'].unique() 

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [10]:
#  Посмотрим уникальные значения для количества детей в семье


df['children'].unique() 

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [11]:
#  Посмотрим уникальные значения на семейный статус 

df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [12]:
#  Посмотрим уникальные значения для типа образования 

df['education'].unique() 

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [13]:
#  Посмотрим уникальные значения цель кредита 

df['purpose'].unique() 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [14]:
# Посмотрим на минимальный и максимальный стаж работы заемщков

df['days_employed'].min()

-18388.949900568383

In [15]:
df['days_employed'].max()

401755.40047533

# Основные выводы по данным:

Данные содержат 21525 строк. Наименование столбцов сохранено. Доля пропущенных значений составляет 20 %, удаление пропусков приведет к значительной потери информации. Значения пропущены в столбцах `days_employed` = 2174, `total_income` = 2174.  Скорее всего, заемщики с пропущенными значениями брали "быстрый" кредит, без предоставления подтверждающих доход документов. Удалять информацию о таких клиентах считаем не целесообразно. 
Доля дублирующих значений не значительна, удалим данные.
Для удобство работы было принято решение привести столбец с данными `days_employed` к типу данных int. Обнаружены отрицательные значения в колонке `days_employed` и в столбце `children`. 



### Обработка пропущенные значения

In [16]:
# Пропущенные значения  в столбцах days_employed и total_income заменяем на ноль

df = df.fillna(0)

In [17]:
df.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [18]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Заменены пропуски в столбцах: days_employed и total_income с Nan на 0, чтобы в дальнейшем можно использовать в арифметических действиях. 
Скорее всего, заемщики с пропущенными значениями брали "быстрый" кредит, без предоставления подтверждающих доход документов, поэтому информации нет. Удалять информацию о таких клиентах считаем не целесообразно. 


### Обработка дубликаты

In [19]:
df = df.drop_duplicates()

In [20]:
df.duplicated().sum()

0

Доля дублирующих значений не значительна, уменьше 1 %, удаление не навредить исследованию.

### Обработка артефактов в данных

In [21]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [22]:
df.loc[df['children'] == -1,'children'] = 1 # замена некореектного ввода "-1"

In [23]:
df.loc[df['children'] == 20,'children'] = 2 # замена некореектного ввода "20"

In [24]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

В столбце `children` было замечено не корректно занесенная информация. Элементы: "-1" и "20" 'заменена на "1" и "2"

### Замена типа данных

In [25]:
df['days_employed']  = df['days_employed'].astype('int') # замена типа данных

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 12 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null float64
purpose             21471 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


Для изменения данных выбран метод astype() c аргументом int. Чтобы изменить вещественный тип данных на целочисленный

### Лемматизация

In [27]:
# Для получение лемматизатора для слов вызовем библиотеку pymystem3 

from pymystem3 import Mystem
m = Mystem()

df['lemmas'] = df.purpose.map(m.lemmatize) # Лемматизация строки


In [28]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [29]:
#Создание функции для категоризации целей кредита

def purpose_category(row):
    if 'автомобиль' in row['lemmas']:
        return 'автомобиль'
    if 'свадьба' in row['lemmas']:
        return 'свадьба'
    if 'жилье' in row['lemmas']:
        return 'Жилая недвижимость'
    if 'собственное' in row['lemmas']:
        return 'Жилая недвижимость'
    if 'недвижимость' in row['lemmas']:
        return 'Коммерческая недвижимость'
    if 'образование' in row['lemmas']:
        return 'Образование'  

In [30]:
df['category_purpose'] = df.apply(purpose_category, axis = 1) # создание нового столбца и применение новой функции


In [31]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,category_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",Жилая недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",Жилая недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",Образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


Выделены леммы в значениях столбца с целями получения кредита, с помощью Лемматизации были отобраны категории целей кредитов и создан новый столбец `category_purpose`.

## Исследовательский анализ данных

### Зависимость между наличием детей и возвратом кредита в срок

In [32]:
children_pivot = df.pivot_table(index=['children'], columns=['debt'], values='purpose', aggfunc='count')

In [33]:
children_pivot['total'] = children_pivot[1] + children_pivot[0]

In [34]:
children_pivot['share_debtor'] = children_pivot[1] / children_pivot['total'] *100

In [35]:
children_pivot.sort_values(by = 'share_debtor',ascending = False)

debt,0,1,total,share_debtor
children,,,,
4,37.0,4.0,41.0,9.756098
2,1926.0,202.0,2128.0,9.492481
1,4411.0,445.0,4856.0,9.163921
3,303.0,27.0,330.0,8.181818
0,13044.0,1063.0,14107.0,7.535266
5,9.0,NaN,NaN,NaN


Вероятность невозврата кредита выше у людей c детьми

### Зависимость между семейным положением и возвратом кредита в срок

In [36]:
family_pivot = df.pivot_table(index=['family_status'], columns=['debt'], values='purpose', aggfunc='count')

In [37]:
family_pivot['total'] = family_pivot[1] + family_pivot[0]

In [38]:
family_pivot['share_debtor'] = family_pivot[1] / family_pivot['total'] * 100

In [39]:
family_pivot.sort_values(by = 'share_debtor',ascending = False)

debt,0,1,total,share_debtor
family_status,,,,
Не женат / не замужем,2536,274,2810,9.750890
гражданский брак,3775,388,4163,9.320202
женат / замужем,11413,931,12344,7.542126
в разводе,1110,85,1195,7.112971
вдовец / вдова,896,63,959,6.569343


Вероятность возникновения долга у незамужних заемщиков выше, чем у замужних.  Самый низкий показатель у категории - вдовец/вдова

### Зависимость между уровнем дохода и возвратом кредита в срок

In [40]:
df['category_income'] = pd.qcut(df['total_income'],5)

In [41]:
income_pivot = df.pivot_table(index=['category_income'], columns=['debt'], values='purpose', aggfunc='count')

In [42]:
income_pivot['total'] = income_pivot[0] + income_pivot[1]

In [43]:
income_pivot['share_debtor'] = income_pivot[1] / income_pivot['total'] * 100

In [44]:
income_pivot.sort_values(by = 'share_debtor',ascending = False)

debt,0,1,total,share_debtor
category_income,,,,
"(116322.861, 156400.32]",3919,375,4294,8.733116
"(156400.32, 214546.071]",3923,371,4294,8.639963
"(79057.107, 116322.861]",3930,364,4294,8.476945
"(-0.001, 79057.107]",3964,331,4295,7.706636
"(214546.071, 2265604.029]",3994,300,4294,6.986493


Доля должников примерно одинакова во всех категориях дохода выделяется только в категории с высоким доходом, доля просроченной задолженности в этой категории меньше всего. 

### Влияние целей кредита на его возврат в срок

In [45]:
purpose_pivot = df.pivot_table(index=['category_purpose'], columns=['debt'], values='income_type', aggfunc='count')

In [46]:
purpose_pivot['total'] = purpose_pivot[1] + purpose_pivot[0]

In [47]:
purpose_pivot['share_debtor'] = purpose_pivot[1] / purpose_pivot['total'] * 100

In [48]:
purpose_pivot.sort_values(by = 'share_debtor',ascending = False)

debt,0,1,total,share_debtor
category_purpose,,,,
автомобиль,3905,403,4308,9.354689
Образование,3644,370,4014,9.217738
свадьба,2149,186,2335,7.965739
Коммерческая недвижимость,5879,474,6353,7.461042
Жилая недвижимость,4153,308,4461,6.904282


Вероятность возникновения просрочки у кредитов с целью - "Автомобиль" и "Образование" выше всего.
Меньше просрочек с целью "Жилая недвижимость"


# Общий вывод

В ходе исследования на действующей базе клиентов, состоящих из 21471 респондентов было выявлено, что при построении модели кредитного скоринга следует с большим риском относиться к потенциальным заемщикам цель кредита, которых покупка автомобиля или оплата образования. 
Также вероятность возникновения долга у незамужних выше, чем возникновения долга у замужних, при этом самый низкий показатель у людей в разводе. В свою очередь доля должников выше у людей с детьми. 
Что касается уровня дохода доля должников примерно одинакова во всех категориях, выделяется только в категория клиентов с высоким доходом, просроченной задолженности там меньше всего.
